# 06 – OSM Sundarbans


**Import libraries**

In [6]:
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
import contextily as cx
import geopandas as gpd
from shapely.geometry import box
from pathlib import Path
from ceam_config import BBOX, PATHS, CRS_METRIC, CRS_WGS84

In [ ]:
in_dir = Path("../outputs/admin")

region_3857 = gpd.read_file(in_dir / "countries_3857.geojson")
bbox_3857   = gpd.read_file(in_dir / "bbox_3857.geojson")

**Terminal commands to get OSM Map of Sundarban Boundary**

```bash
npm install -g osmtogeojson

REL_ID=14937802
mkdir -p data/basemaps/osm

curl -G "https://overpass-api.de/api/interpreter" \
  --data-urlencode "data=
[out:json][timeout:180];
relation(${REL_ID});
(._;>;);
out body;
" \
| osmtogeojson \
> data/basemaps/osm/sundarbans_relation_${REL_ID}.geojson
```

**Load and Keep Only Polygon**

In [4]:
sund = gpd.read_file("../data/basemaps/osm/sundarbans_relation_14937802.geojson")

# Keep only polygons (drop lines/points)
sund_poly = sund[sund.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()

print("Geoms:", sund.geometry.type.value_counts().to_dict())
print("Poly geoms:", sund_poly.geometry.type.value_counts().to_dict())
print("Poly bounds:", sund_poly.total_bounds)


Geoms: {'LineString': 34, 'Point': 6, 'Polygon': 1}
Poly geoms: {'Polygon': 1}
Poly bounds: [88.676409  21.5401969 89.9354946 22.5017564]


In [5]:
# Project to 3857 for tiles
region_3857 = region.to_crs(epsg=3857)
bbox_3857 = bbox_gdf.to_crs(epsg=3857)
sund_3857 = sund_poly.to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(10, 8))

# Set extent first
xmin, ymin, xmax, ymax = bbox_3857.total_bounds
pad_x = (xmax - xmin) * 0.18   # slightly more breathing room
pad_y = (ymax - ymin) * 0.18
ax.set_xlim(xmin - pad_x, xmax + pad_x)
ax.set_ylim(ymin - pad_y, ymax + pad_y)

# Basemap (keep Mapnik, but let it choose the right zoom automatically)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)

# Context border (make it subtle so it doesn't fight the Sundarbans boundary)
region_3857.boundary.plot(ax=ax, color="#111827", linewidth=1.2, alpha=0.35, zorder=3)

# Sundarbans polygon overlay (hero outline)
sund_3857.plot(
    ax=ax,
    facecolor="none",
    edgecolor="#E11D48",
    linewidth=3.5,
    zorder=10
)

# Bbox (slightly quieter)
bbox_3857.boundary.plot(
    ax=ax,
    color="#1D4ED8",
    linewidth=1.6,
    linestyle="--",
    alpha=0.75,
    zorder=5
)

ax.set_title("CEAM – Sundarbans boundary (OSM relation) over OSM basemap")
ax.set_axis_off()
plt.tight_layout()
plt.savefig("../outputs/figures/ceam_sundarbans_admin_hero.png", dpi=300, bbox_inches="tight")
plt.show()


NameError: name 'region' is not defined

In [9]:
print("AOI bbox (EPSG:4326):", bbox)
print("bbox_3857 bounds:", bbox_3857.total_bounds)
print("sund_3857 bounds:", sund_3857.total_bounds)
print("region_3857 bounds:", region_3857.total_bounds)


AOI bbox (EPSG:4326): {'lon_min': 88.5, 'lat_min': 21.5, 'lon_max': 89.92, 'lat_max': 22.5}
bbox_3857 bounds: [ 9851774.93520471  2451599.08737786 10009848.61213116  2571663.04717365]
sund_3857 bounds: [ 9871412.69525606  2456409.09675154 10011573.4631132   2571874.67952194]
region_3857 bounds: [ 7589389.42046348   889589.55841096 10842803.54553966  4231219.37264593]
